In [2]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
#torch.backends.cudnn.enabled = False


In [3]:
manualSeed=187
fsize=64
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

torch.cuda.manual_seed_all(manualSeed)

cudnn.benchmark = True

dataroot='batched_dataset_64/mat_files/'
path=dataroot

Random Seed:  187


In [4]:
import os
import scipy.io
ls=os.listdir(path)
#print ls

In [5]:
#xs=[]
#ys=[]
#for i in ls:
    #im1 = scipy.io.loadmat(path+i)
    #im1=im1['im']
    #xs.append(im1.shape[0])
    #ys.append(im1.shape[1])

In [6]:
#s1=min(xs)
#s2=min(ys)
s1=64
s2=64
print (s1)
print (s2)

64
64


In [7]:
imageSize=(s1,s2,3)
ls=ls[0:1000]
lls=len(ls)
import numpy as np
X=np.zeros((lls,3,s1,s2))
Y=np.zeros((lls,3,s1,s2))
count=0
import Image
for idx,i in enumerate(ls):
    li=len(i)
    i=i[0:li-4]
    im1 = scipy.io.loadmat(path+i+'.mat')
    im2=np.asarray(Image.open('batched_dataset_64/image_files/'+i+'.png'))    
    im2=im2/255.0;
    im1=im1['im']

    X[count,0,:,:]=im1[ 0:s1    , 0:s2 ,0]
    X[count,1,:,:]=im1[ 0:s1    , 0:s2 ,1]
    X[count,2,:,:]=im1[ 0:s1    , 0:s2 ,2]

    Y[count,0, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,0]
    Y[count,1, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,1]
    Y[count,2, 0:s1    , 0:s2 ]=im2[ 0:s1    , 0:s2 ,2]
    count=count+1
    
 

In [8]:
   
print(X.shape)

(1000, 3, 64, 64)


In [9]:
#time pass
def convert_channels(a):
    b=np.zeros((a.shape[1],a.shape[2],3))
    b[:,:,0]=a[0,:,:]
    b[:,:,1]=a[1,:,:]
    b[:,:,2]=a[2,:,:]
    return b   

def convert_channels2(a):
    b=np.zeros((3,a.shape[1],a.shape[2]))
    b[0,:,:]=a[:,:,0]
    b[1,:,:]=a[:,:,1]
    b[2,:,:]=a[:,:,2]
    return b   



In [10]:
'''
import scipy.io as sio
lx=X.shape[0]
imgs=np.zeros(X.shape)
for i in range(lx):
    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})
    a=X[i,:,:,:]
    im1_8bit = np.clip(a*255, 0, 255).astype('uint8')
    imgs[i,:,:,:]=( im1_8bit)
'''

"\nimport scipy.io as sio\nlx=X.shape[0]\nimgs=np.zeros(X.shape)\nfor i in range(lx):\n    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})\n    a=X[i,:,:,:]\n    im1_8bit = np.clip(a*255, 0, 255).astype('uint8')\n    imgs[i,:,:,:]=( im1_8bit)\n"

In [11]:
'''
# save the training set
path='batched_hdr/'
path2='batched_hdr_tone/'
import scipy.io as sio
lx=X.shape[0]
for i in range(lx):
    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})
    see_image(X[i,:,:,:],i,path2)
'''

"\n# save the training set\npath='batched_hdr/'\npath2='batched_hdr_tone/'\nimport scipy.io as sio\nlx=X.shape[0]\nfor i in range(lx):\n    sio.savemat(path+str(i)+'.mat',{'im':X[i,:,:,:]})\n    see_image(X[i,:,:,:],i,path2)\n"

In [12]:
print (X.shape)
#Z=np.zeros((X.shape[0],3,X.shape[1],X.shape[2]))
#Z[:,0,:,:]=X[:,:,:,0]
#Z[:,1,:,:]=X[:,:,:,1]
#Z[:,2,:,:]=X[:,:,:,2]
#X=Z

(1000, 3, 64, 64)


In [13]:
batchSize=10
workers=1
def get_batch():
    import numpy.random
    pos=numpy.random.randint(0,X.shape[0],(batchSize))
    x1=X[pos]
    y1=Y[pos]
    return (x1), (y1)



ngpu = 1
nz = 100
ngf = 64
ndf = 64
nc = 3

In [14]:
#for i in dataloader:
#    print (i)

In [15]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [16]:
#class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        model=[]
        xx=[]
        #model.append(nn.Conv2d(nc, ndf, 3, 2, 0, bias=False))
        xx.append(nn.Conv2d(nc, ndf, 3, 1, 1, bias=False))
        self.v1=64
        model.append(nn.LeakyReLU(0.2, inplace=True) )
        model.append(nn.Conv2d(ndf, ndf * 2, 3, 1, 1, bias=False))
        #self.bn1=nn.BatchNorm2d(ndf * 2)
        model.append(nn.LeakyReLU(0.2, inplace=True))
        model.append(nn.Conv2d(ndf * 2, ndf*4, 3, 1, 1, bias=False))
        #self.bn2=nn.BatchNorm2d(ndf * 4)
        model.append(nn.LeakyReLU(0.2, inplace=True))
        model.append(nn.Conv2d(ndf*4, 3, 3, 1, 1, bias=False))
        model.append(nn.Sigmoid())

        self.model = nn.Sequential(*model)
        self.xx = nn.Sequential(*xx)
        

        
    def forward(self, input):
        #print (input)
        op=self.xx(input)
        output = self.model(op)


        return output


In [17]:
netG = _netG(ngpu)
netG.apply(weights_init)
str_netG=''
#str_netG='netG_epoch_1.pth'
if str_netG != '':
    netG.load_state_dict(torch.load(str_netG))
print(netG)

_netG (
  (model): Sequential (
    (0): LeakyReLU (0.2, inplace)
    (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): LeakyReLU (0.2, inplace)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(256, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): Sigmoid ()
  )
  (xx): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
)


In [ ]:
cuda=True
lr=0.0002
beta1=0.5
niter=800000
outf='output_cnn'
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize[0], imageSize[1])

netG.cuda()
criterion.cuda()
input = input.cuda()
input = Variable(input)
criterion=nn.MSELoss()
criterion.cuda()
ss=Y.shape[0]
# setup optimizer
optimizer = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
import torch.nn
for epoch in range(niter):
    #print (epoch,1)
    data, target = get_batch()
    #print (epoch,1)
    vam=np.max(data)
    vami=np.min(data)
    data=data/vam

    data, target= torch.from_numpy(data), torch.from_numpy(target)
    data, target = data.float().cuda(), target.float().cuda()
    data, target = Variable(data), Variable(target)
    #optimizer.zero_grad()
    #print (data)
    #output = netG(data)
    #print (output)
    #print (target)

    #print (output.size())
    #print(target.size())
    loss = criterion(data, target)
    #loss.backward()
    #optimizer.step()
    #print (epoch,2)
    if epoch % 300 == 0:
        xyvap=np.random.randint(ss)
        #vutils.save_image(Y[xyvap],
        #        '%s/real_samples.png' % outf)

        ddv=X[xyvap:xyvap+1]
        ddv=ddv/np.max(ddv)
        ddv=ddv*255
        #print (ddv.shape)
        ddv2=Variable(torch.from_numpy(ddv).float().cuda())
        
        ddv3=Y[xyvap:xyvap+1]
        ddv3=ddv3*255
        #print (ddv.shape)
        ddv4=Variable(torch.from_numpy(ddv3).float().cuda())
        #print (ddv2)

        #fake = netG(ddv2)
        #print (fake.data)
        #print (ddv4)

        vutils.save_image(ddv2.data,
                '%s/samples_epoch_%03d.png' % (outf, epoch) , normalize=True)
        vutils.save_image(ddv4.data,
                '%s/samples_epoch_%03d.png' % (outf, epoch+1), normalize=True)

    if(epoch%100==0):
        print(epoch,loss.data[0])
    if(epoch%1000==0): 
        torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))

0 0.135694012046
100 0.185374602675
200 0.279772937298
300 0.276449739933
400 0.149568632245
500 0.229862824082
600 0.247276470065
700 0.162724196911
800 0.292690813541
900 0.246632367373
1000 0.296729564667
1100 0.200272843242
1200 0.183077558875
1300 0.158477306366
1400 0.316751658916
1500 0.272574931383
1600 0.0920715332031
1700 0.110912941396
1800 0.152114078403
1900 0.154284715652
2000 0.128168523312
2100 0.180111184716
2200 0.214414656162
2300 0.146823182702
2400 0.177805840969
2500 0.0849889814854
2600 0.208776757121
2700 0.272645801306
2800 0.209274366498
2900 0.297869801521
3000 0.20243626833
3100 0.231815680861
3200 0.234636843204
3300 0.131128534675
3400 0.0722456499934
3500 0.185280352831
3600 0.336105912924
3700 0.248631179333
3800 0.160547286272
3900 0.18820720911
4000 0.15023213625
4100 0.215668991208
4200 0.175281584263
4300 0.262294888496
4400 0.244627520442
4500 0.16038608551
4600 0.0888355895877
4700 0.152383729815
4800 0.284998416901
4900 0.17831864953
5000 0.231642

39900 0.219248145819
40000 0.206449717283
40100 0.249788984656
40200 0.243732690811
40300 0.172553300858
40400 0.127497777343
40500 0.132267445326
40600 0.245131030679
40700 0.337338358164
40800 0.215449094772
40900 0.198129162192
41000 0.147881746292
41100 0.192114129663
41200 0.0531369931996
41300 0.229848891497
41400 0.165535956621
41500 0.161345735192
41600 0.205354750156
41700 0.251670539379
41800 0.167603567243
41900 0.263697057962
42000 0.314040124416
42100 0.11489675194
42200 0.301216423512
42300 0.232469245791
42400 0.232873231173
42500 0.179581031203
42600 0.184978455305
42700 0.188115820289
42800 0.284292727709
42900 0.195615440607
43000 0.191403657198
43100 0.357591331005
43200 0.235363140702
43300 0.128481253982
43400 0.162432387471
43500 0.11630629003
43600 0.186683446169
43700 0.127618521452
43800 0.164946600795
43900 0.199167236686
44000 0.188414648175
44100 0.284476846457
44200 0.151152297854
44300 0.164118617773
44400 0.189339742064
44500 0.129961952567
44600 0.246961